
# Datawhale 零基础入门数据挖掘-Task1 赛题理解

赛题：零基础入门数据挖掘 - 二手车交易价格预测
地址：https://tianchi.aliyun.com/competition/entrance/231784/introduction?spm=5176.12281957.1004.1.38b02448ausjSX




## 1.赛题背景
赛题以预测二手车的交易价格为任务，该数据来自某交易平台的二手车交易记录，总数据量超过40w，包含31列变量信息，其中15列为匿名变量。为了保证比赛的公平性，将会从中抽取15万条作为训练集，5万条作为测试集A，5万条作为测试集B，同时会对name、model、brand和regionCode等信息进行脱敏。



## 2.数据概况

---
数据官方解释

---
**train.csv**
* name - 汽车编码
* regDate - 汽车注册时间
* model - 车型编码
* brand - 品牌
* bodyType - 车身类型
* fuelType - 燃油类型
* gearbox - 变速箱
* power - 汽车功率
* kilometer - 汽车行驶公里
* notRepairedDamage - 汽车有尚未修复的损坏
* regionCode - 看车地区编码
* seller - 销售方
* offerType - 报价类型
* creatDate - 广告发布时间
* price - 汽车价格
* v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14'（根据汽车的评论、标签等大量信息得到的embedding向量）【人工构造 匿名特征】
　
 
数字全都脱敏处理，都为label encoding形式，即数字形式

### 2.1数据格式化（预处理）
下载数据后使用pandas读取数据发现数据未完成格式化，许多总数据被合并为一列，需要将数据分列分开

### 代码示例

#### 导入数据

In [16]:

import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

df = pd.read_csv('used_car_train_20200313.csv')
df2 = pd.read_csv('used_car_testA_20200313.csv')

In [10]:
#训练集数据
df.head(3)

,SaleID name regDate model brand bodyType fuelType gearbox power kilometer notRepairedDamage regionCode seller offerType creatDate price v_0 v_1 v_2 v_3 v_4 v_5 v_6 v_7 v_8 v_9 v_10 v_11 v_12 v_13 v_14
0,0 736 20040402 30.0 6 1.0 0.0 0.0 60 12.5 0.0 ...
1,1 2262 20030301 40.0 1 2.0 0.0 0.0 0 15.0 - 43...
2,2 14874 20040403 115.0 15 1.0 0.0 0.0 163 12.5...


In [11]:
#测试集数据
df2.head(3)

,SaleID name regDate model brand bodyType fuelType gearbox power kilometer notRepairedDamage regionCode seller offerType creatDate v_0 v_1 v_2 v_3 v_4 v_5 v_6 v_7 v_8 v_9 v_10 v_11 v_12 v_13 v_14
0,150000 66932 20111212 222.0 4 5.0 1.0 1.0 313 ...
1,150001 174960 19990211 19.0 21 0.0 0.0 0.0 75 ...
2,150002 5356 20090304 82.0 21 0.0 0.0 0.0 109 7...


#### 数据分离

In [12]:
#训练集分离部分
r_df = []
columns = df.columns[0].split(' ')
for i in tqdm(range(df.shape[0])):
    temp = []
    temp = ' '.join(str(i) for i in df.iloc[i]).split(' ')
    r_df.append(copy.copy(temp))
r_df = pd.DataFrame(r_df)
r_df.columns = columns
r_df.index = r_df['SaleID']
r_df = r_df.drop(r_df.columns.values.tolist()[0],axis=1)
r_df.to_csv('formatted_used_car_train_20200313.csv')
r_df.head(3)

100%|██████████| 150000/150000 [00:09<00:00, 15586.45it/s]


,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,...,0.23567590669911015,0.10198824077953883,0.129548661418789,0.02281636740006269,0.09746182870576199,-2.8818032385553165,2.8040967707208506,-2.4208207926122784,0.7952919433118377,0.9147624995703408
1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,...,0.2647772555037097,0.12100359404116512,0.1357307068829055,0.026597448118262774,0.020581662632484482,-4.9004818817666775,2.0963376444273414,-1.0304828371563102,-1.7226737753851349,0.2455224109670493
2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,...,0.25141014780875875,0.11491227654046415,0.16514749334496415,0.06217283730726245,0.02707482416830506,-4.846749260269903,1.803558941229932,1.5653296250457633,-0.8326873267265079,-0.22996285613259074


In [14]:
#测试集分离部分
r_df = []
columns = df2.columns[0].split(' ')
for i in tqdm(range(df2.shape[0])):
    temp = []
    temp = ' '.join(str(i) for i in df2.iloc[i]).split(' ')
    r_df.append(copy.copy(temp))
r_df = pd.DataFrame(r_df)
r_df.columns = columns
r_df.index = r_df['SaleID']
r_df = r_df.drop(r_df.columns.values.tolist()[0],axis=1)
r_df.to_csv('formatted_used_car_testA_20200313.csv')
r_df.head(3)

100%|██████████| 50000/50000 [00:03<00:00, 15763.50it/s]


,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
150000,66932,20111212,222.0,4,5.0,1.0,1.0,313,15.0,0.0,...,0.26440452462313,0.12179975526236647,0.07089899944208154,0.1065583474183833,0.07886720124806819,-7.050968898903568,-0.8546256772168985,4.8001508741748085,0.6200105915220468,-3.664654280846407
150001,174960,19990211,19.0,21,0.0,0.0,0.0,75,12.5,1.0,...,0.26174517256607843,0.0,0.09673306358066952,0.013704976264260557,0.052382729107606815,3.6794175097636237,-0.7290394415567809,-3.796106922616791,-1.5412303833299437,-0.7570547294745845
150002,5356,20090304,82.0,21,0.0,0.0,0.0,109,7.0,0.0,...,0.2602159826726613,0.11208114365646027,0.0780824983885986,0.0620780188429273,0.050540421383761185,-4.926689857485238,1.001106476775805,0.8265624246176062,0.13822636745570258,0.7540328013603533


### 2.2数据观察
观察数据中的出现频次与种类，深入了解特征特性

In [18]:
#训练集观察
file_path = ''
train_file = 'formatted_used_car_train_20200313.csv'
pd.set_option('display.max_columns', None)
df_train = pd.read_csv(file_path+train_file,header=0,index_col=0)
df_view = [[],[]]
columns = []
for i in range(df_train.shape[1]):
    groupby_name = df_train.columns[i]
    if groupby_name == 'price':
        continue
    assert groupby_name!='price'
    columns.append(groupby_name)
    #单个特征最大出现次数
    df_view[0].append(copy.copy(df_train.groupby(groupby_name)['price'].count().max()))
    #特征类总类型数量
    df_view[1].append(copy.copy(df_train.groupby(groupby_name)['price'].count().count()))
df_view = pd.DataFrame(df_view)
df_view.columns = columns
df_view.index = ['max count','numbers of index']
df_view.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
max count,282,180,11762,31480,41420,91656,111623,12829,96877,111361,369,149999,150000,5848,20,20,20,20,20,4485,35465,5467,1597,3486,20,20,20,20,20
numbers of index,99662,3894,248,40,8,7,2,566,13,3,7905,2,1,96,143997,143998,143997,143998,143998,139624,109766,138709,142451,140617,143997,143997,143997,143998,143998


In [27]:
#测试集观察
test_file = 'formatted_used_car_testA_20200313.csv'
df_test = pd.read_csv(file_path+test_file,header=0,index_col=0)
df_view = [[],[]]
columns = []
p = np.array(['s']*df_test.shape[0])
p.shape=[-1,1]
p = pd.DataFrame(p)
p.index = df_test.index
p.columns = ['price']
df_test = pd.concat([df_test,p],sort=False)
df_test['price'] = p
for i in range(df_test.shape[1]):
    groupby_name = df_test.columns[i]
    if groupby_name == 'price':
        continue
    assert groupby_name!='price'
    columns.append(groupby_name)
    #单个特征最大出现次数
    df_view[0].append(copy.copy(df_test.groupby(groupby_name)['price'].count().max()))
    #特征类总类型数量
    df_view[1].append(copy.copy(df_test.groupby(groupby_name)['price'].count().count()))
df_view = pd.DataFrame(df_view)
df_view.columns = columns
df_view.index = ['max count','numbers of index']
df_view.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
max count,97,60,3896,10348,13985,30656,37301,4195,32189,37249,146,50000,50000,1966,10,10,10,10,10,1408,11706,1855,566,1128,10,10,10,10,10
numbers of index,37453,3835,247,40,8,7,2,445,13,3,6971,1,1,69,49186,49186,49186,49186,49186,47788,37624,47355,48634,48063,49186,49186,49186,49186,49186


### 2.3数据观察结果总结
经过观察发现部分特征在描述中有未解释清楚地方
notReparedDamage除了描述中的取值[0,1]以外，还有缺失值的情况
seller和offerType取值几乎完全恒定，由于另一取值样本过少或者没有，所以可能无法推断这两个特征与price之间关系

## 3.预测指标

**本赛题的评价标准为MAE(Mean Absolute Error):**

$$
MAE=\frac{\sum_{i=1}^{n}\left|y_{i}-\hat{y}_{i}\right|}{n}
$$
其中$y_{i}$代表第$i$个样本的真实值，其中$\hat{y}_{i}$代表第$i$个样本的预测值。


## 4.代码示例

In [ ]:
#原始数据导入
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

df = pd.read_csv('used_car_train_20200313.csv')
df2 = pd.read_csv('used_car_testA_20200313.csv')

In [ ]:
#训练集预处理部分
r_df = []
columns = df.columns[0].split(' ')
for i in tqdm(range(df.shape[0])):
    temp = []
    temp = ' '.join(str(i) for i in df.iloc[i]).split(' ')
    r_df.append(copy.copy(temp))
r_df = pd.DataFrame(r_df)
r_df.columns = columns
r_df.index = r_df['SaleID']
r_df = r_df.drop(r_df.columns.values.tolist()[0],axis=1)
r_df.to_csv('formatted_used_car_train_20200313.csv')

In [ ]:
#测试集预处理部分
r_df = []
columns = df2.columns[0].split(' ')
for i in tqdm(range(df2.shape[0])):
    temp = []
    temp = ' '.join(str(i) for i in df2.iloc[i]).split(' ')
    r_df.append(copy.copy(temp))
r_df = pd.DataFrame(r_df)
r_df.columns = columns
r_df.index = r_df['SaleID']
r_df = r_df.drop(r_df.columns.values.tolist()[0],axis=1)
r_df.to_csv('formatted_used_car_testA_20200313.csv')

In [ ]:
#训练集观察
file_path = ''
train_file = 'formatted_used_car_train_20200313.csv'
pd.set_option('display.max_columns', None)
df_train = pd.read_csv(file_path+train_file,header=0,index_col=0)
df_view = [[],[]]
columns = []
for i in range(df_train.shape[1]):
    groupby_name = df_train.columns[i]
    if groupby_name == 'price':
        continue
    assert groupby_name!='price'
    columns.append(groupby_name)
    #单个特征最大出现次数
    df_view[0].append(copy.copy(df_train.groupby(groupby_name)['price'].count().max()))
    #特征类总类型数量
    df_view[1].append(copy.copy(df_train.groupby(groupby_name)['price'].count().count()))
df_view = pd.DataFrame(df_view)
df_view.columns = columns
df_view.index = ['max count','numbers of index']
df_view.head()

In [ ]:
#测试集观察
test_file = 'formatted_used_car_testA_20200313.csv'
df_test = pd.read_csv(file_path+test_file,header=0,index_col=0)
df_view = [[],[]]
columns = []
p = np.array(['s']*df_test.shape[0])
p.shape=[-1,1]
p = pd.DataFrame(p)
p.index = df_test.index
p.columns = ['price']
df_test = pd.concat([df_test,p],sort=False)
df_test['price'] = p
for i in range(df_test.shape[1]):
    groupby_name = df_test.columns[i]
    if groupby_name == 'price':
        continue
    assert groupby_name!='price'
    columns.append(groupby_name)
    #单个特征最大出现次数
    df_view[0].append(copy.copy(df_test.groupby(groupby_name)['price'].count().max()))
    #特征类总类型数量
    df_view[1].append(copy.copy(df_test.groupby(groupby_name)['price'].count().count()))
df_view = pd.DataFrame(df_view)
df_view.columns = columns
df_view.index = ['max count','numbers of index']
df_view.head()

In [28]:
#评价指标
# coding=utf-8
import numpy as np
from sklearn import metrics

def my_mse(y_true,y_pred):
    return metrics.mean_squared_error(y_true, y_pred)